# ECommerce AI Trends

How modern day AI models (LLMs, Diffusion Models, & Embedding Models) are starting to be applied to E-Commerce Scenarios

1. Trying things on with CATVTON
2. Embedding/LLM based reccomendation algorithms
3. Review summarization
4. Copilot for Shopping
5. Product placement across scene via Flux Lora

---
# Virtual Try On via Diffusion Models

Virtual try-on is a technology that allows people to see how clothing or accessories might look on them without physically wearing the items. It uses image generation (diffusion) models to align and adjust the garment images to fit the person's body shape and pose. We have seen examples using Augmented Reality, however this allows a much more realistic generation.

<img src="./CatVTON/supreme_jacket_output.png" width=600>

This example was generated using a recently released virtual try on model [CatVTON](https://arxiv.org/abs/2407.15886)- an optimized model based on [Stable Diffusion v1.5 Inpainting](https://huggingface.co/runwayml/stable-diffusion-inpainting), allowing quick and lightweight masked diffusion model generation 

<img src="./CatVTON/catvton_diagram.png" width=600>

### [Use Via HuggingFace Spaces](https://huggingface.co/spaces/zhengchong/CatVTON)  
### [Use Via Hosted Gradio Interface](http://120.76.142.206:8888/)

---
# Personalized and Dynamic Product Photos via Diffusion Models

https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/README_flux.md

flux lora hehe

---
# Embedding Model Based Product Reccomendation

---
# LLM Shopping Copilot

---
# Review and Product Summarizations 

Google shopping experience these days, amazon reviews